In [48]:
from dataclasses import dataclass
import numpy as np
from typing import List


def random_graph(n: int, p: float = 0.25):
    matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(i + 1, n):
            if np.random.rand() < p:
                matrix[i, j] = 1
                matrix[j, i] = 1
    return matrix


@dataclass
class Agent:
    licenses: List[int]
    bids: List[List[float]]
    #crossBids: List[List[float]]


def generate(regions: int = 12, 
             blocks: int = 16, 
             agents: int = 20,
             T: float = 1.5):
    matrix = random_graph(regions)
    remains = [blocks] * regions
    A = []
    for agent in range(agents):
        licenses = [0] * regions
        for region in range(regions):
            n = remains[region]
            probabilities = np.exp(-np.arange(n + 1) / T)
            probabilities /= np.sum(probabilities)
            l = np.random.choice(np.arange(n + 1), p=probabilities)
            remains[region] -= l
            licenses[region] = l
        bids = [[]] * regions
        for region in range(regions):
            l = licenses[region]
            if l != 0:
                s = blocks + 1 - l
                bids[region] = (np.random.rand(s) <= 0.6) * np.random.rand(s) * 10 ** 6
        #### сгенерируй межрегиональные ставки
        A.append(Agent(licenses, bids))
    return A

A = generate()
sorted([sum(a.licenses) for a in A])


[2, 4, 4, 5, 5, 5, 6, 7, 7, 8, 9, 9, 11, 11, 13, 14, 15, 15, 16, 20]